## 1.  The simplest way to create and manage threads in Python is using the threading.Thread class.

In [1]:
import threading


def worker():
    print("Worker thread is running")


if __name__ == "__main__":
    t = threading.Thread(target=worker)
    t.start()  # Start the thread
    t.join()  # Wait for the thread to finish

Worker thread is running


## 2. Passing Arguments to Threads

In [2]:
import threading


def worker(num):
    print(f"Worker {num} started")


if __name__ == "__main__":
    threads = []
    for i in range(5):
        t = threading.Thread(target=worker, args=(i,))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()  # Wait for all threads to finish

Worker 0 started
Worker 1 started
Worker 2 started
Worker 3 started
Worker 4 started


## 3. You can also create a subclass of threading.Thread to define thread behavior.

In [3]:
import threading


class MyThread(threading.Thread):
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.name = name

    def run(self):
        print(f"Thread {self.name} is running")


if __name__ == "__main__":
    t1 = MyThread("A")
    t2 = MyThread("B")

    t1.start()
    t2.start()

    t1.join()
    t2.join()

Thread A is running


Thread B is running


## 4. Using concurrent.futures.ThreadPoolExecutor
## ThreadPoolExecutor from the concurrent.futures module is a higher-level interface for managing thread pools, making it easier to run tasks concurrently.

In [5]:
from concurrent.futures import ThreadPoolExecutor
import time


def worker(n):
    print(f"Worker {n} is running")
    time.sleep(1)
    return n * n


if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = [executor.submit(worker, i) for i in range(5)]

    results = [f.result() for f in futures]
    print(f"Results: {results}")

Worker 0 is running
Worker 1 is running
Worker 2 is running
Worker 3 is running
Worker 4 is running
Results: [0, 1, 4, 9, 16]


## 5. Using ThreadPoolExecutor.map()
## map() from ThreadPoolExecutor allows you to apply a function to multiple inputs and run them concurrently.



In [7]:
from concurrent.futures import ThreadPoolExecutor


def worker(n):
    time.sleep(1)
    print(n * n, end=" ")
    return n * n


if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=3) as executor:
        results = executor.map(worker, range(5))
    print(f"Results: {list(results)}")

1 0 4 9 16 Results: [0, 1, 4, 9, 16]


## 6. Thread Synchronization with threading.Lock
To prevent race conditions when multiple threads access shared resources, you can use a Lock. A Lock ensures that only one thread at a time can execute a critical section of code.



In [8]:
import threading

lock = threading.Lock()
counter = 0


def increment():
    global counter
    for _ in range(100000):
        with lock:  # Ensure only one thread accesses the critical section at a time
            counter += 1


if __name__ == "__main__":
    threads = []
    for i in range(5):
        t = threading.Thread(target=increment)
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    print(f"Final counter: {counter}")

Final counter: 500000


## 7. Thread Synchronization with threading.Semaphore
A Semaphore allows you to limit the number of threads that can access a shared resource at the same time.



In [9]:
import threading
import time

semaphore = threading.Semaphore(2)  # Only allow 2 threads to access the resource


def worker(name):
    with semaphore:
        print(f"{name} is working")
        time.sleep(1)
    print(f"{name} is done")


if __name__ == "__main__":
    threads = []
    for i in range(5):
        t = threading.Thread(target=worker, args=(f"Worker {i}",))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

Worker 0 is workingWorker 1 is working

Worker 1 is doneWorker 2 is working

Worker 0 is done
Worker 3 is working
Worker 2 is doneWorker 4 is working

Worker 3 is done
Worker 4 is done


## 8. Inter-thread Communication with queue.Queue
You can use queue.Queue for safe communication between threads. It's a thread-safe data structure used to send messages or data between threads.



In [10]:
import threading
import queue


def producer(q):
    for i in range(5):
        q.put(i)  # Put data in the queue
        print(f"Produced {i}")


def consumer(q):
    while True:
        item = q.get()  # Get data from the queue
        if item is None:  # Sentinel value to signal completion
            break
        print(f"Consumed {item}")
        q.task_done()


if __name__ == "__main__":
    q = queue.Queue()

    t1 = threading.Thread(target=producer, args=(q,))
    t2 = threading.Thread(target=consumer, args=(q,))

    t1.start()
    t2.start()

    t1.join()

    q.put(None)  # Send sentinel to stop the consumer
    t2.join()

Produced 0
Produced 1
Produced 2
Produced 3
Produced 4
Consumed 0
Consumed 1
Consumed 2
Consumed 3
Consumed 4


## 9. Thread Synchronization with threading.Event
An Event can be used to signal one or more threads to wait or proceed with execution. It can be set (allow threads to proceed) or cleared (make threads wait).

In [11]:
import threading
import time

event = threading.Event()


def worker():
    print("Waiting for event to be set...")
    event.wait()  # Wait until the event is set
    print("Event received, working...")


def trigger_event():
    time.sleep(2)
    print("Setting event")
    event.set()  # Set the event to allow workers to proceed


if __name__ == "__main__":
    t1 = threading.Thread(target=worker)
    t2 = threading.Thread(target=trigger_event)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

Waiting for event to be set...
Setting event
Event received, working...


## 10. Using threading.Condition for Complex Synchronization
A Condition can be used for more advanced synchronization. Threads can wait for a certain condition to be met before they proceed.

In [12]:
import threading
import time

condition = threading.Condition()
data_ready = False


def producer():
    global data_ready
    with condition:
        print("Producing data...")
        time.sleep(1)
        data_ready = True
        condition.notify()  # Notify a waiting thread that data is ready


def consumer():
    with condition:
        while not data_ready:
            print("Waiting for data...")
            condition.wait()  # Wait until the producer notifies
        print("Data is ready, consuming data")


if __name__ == "__main__":
    t1 = threading.Thread(target=consumer)
    t2 = threading.Thread(target=producer)

    t1.start()
    t2.start()

    t1.join()
    t2.join()

Waiting for data...
Producing data...
Data is ready, consuming data


## 11. Daemon Threads
Daemon threads run in the background and are killed when the main program exits. This is useful for background tasks that don’t need to be completed when the program exits.

In [13]:
import threading
import time


def daemon_task():
    while True:
        print("Daemon task running")
        time.sleep(1)


if __name__ == "__main__":
    t = threading.Thread(target=daemon_task)
    t.daemon = True  # Mark thread as daemon
    t.start()

    time.sleep(3)
    print("Main thread is exiting")

Daemon task running
Daemon task running
Daemon task running
Main thread is exiting


Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
Daemon task running
